## FOR BODY DAMAGE

## IMAGE PRE PROCESSING

## 1. Import The ImageDataGenerator Library

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## :2. Configure ImageDataGenerator Class

##   Image Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

## 3. Apply ImageDataGenerator Functionality To Trainset And Testset

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/training',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/validation',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


## **MODEL BUILDING**

## 1. Importing The Model Building Libraries 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

## 2. Loading The Model

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/training'
valid_path = '/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/validation'

In [ ]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


## 3. Adding Flatten Layer

In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/training/*')

In [ ]:
folders

['/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/training/02-side',
 '/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/training/01-rear',
 '/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/training/00-front']

In [ ]:
x = Flatten()(vgg16.output)

In [ ]:
len(folders)

3

## **4. Adding Output Layer**

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

## **5. Creating A Model Object**

In [ ]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## **6. Configure The Learning Process**

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

## **7. Train The Model**

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 560s 6s/step - loss: 1.2275 - accuracy: 0.5383 - val_loss: 0.8698 - val_accuracy: 0.6608
Epoch 2/25
98/98 [==============================] - 584s 6s/step - loss: 0.7810 - accuracy: 0.7007 - val_loss: 0.8931 - val_accuracy: 0.6491
Epoch 3/25
98/98 [==============================] - 538s 5s/step - loss: 0.4842 - accuracy: 0.8264 - val_loss: 0.8348 - val_accuracy: 0.6842
Epoch 4/25
98/98 [==============================] - 537s 5s/step - loss: 0.3813 - accuracy: 0.8560 - val_loss: 0.9010 - val_accuracy: 0.6901
Epoch 5/25
98/98 [==============================] - 537s 5s/step - loss: 0.2735 - accuracy: 0.8999 - val_loss: 1.0660 - val_accuracy: 0.6901
Epoch 6/25
98/98 [==============================] - 538s 5s/step - loss: 0.2211 - accuracy: 0.9295 - val_loss: 1.0073 - val_accuracy: 0.7076
Epoch 7/25
98/98 [==============================] - 536s 5s/step - loss: 0.2163 - accuracy: 0.9224 - val_loss: 0.9560 - val_accuracy: 0.7251
Epoch 8/25
98

## **8. Save The Model**

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Model/body.h5')

## **9. Test The Model**

In [ ]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [ ]:
model = load_model('/content/drive/MyDrive/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Model/body.h5')

In [ ]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [ ]:
import numpy as np

In [ ]:
data = "/content/drive/MyDrive/IBM - PROJECT/Data set/body-20221023T072112Z-001/body/training/00-front/0008.jpeg"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 0s 498ms/step
front
